In [1]:
import csv
import requests
import datetime
import urllib
#!conda install -y bs4
!pip install bs4
from bs4 import BeautifulSoup
import pandas as pd
# the below gives pandas the ability to grab a html table
#!conda install -y html5lib
import os.path

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
#def get_dist_ids():
#   """Returns the district IDs from the site."""
#   dist_url = 'http://aahar.jharkhand.gov.in/transactions/transactionView'
#   rids = requests.get(dist_url)
#  content = rids.text
#   soup = BeautifulSoup(content, "html.parser")
#  dist_ids = [(option['value'],option.text) for option in soup.find_all('option', value=True)[1:-2]]
#   return dist_ids

In [3]:
#print(get_dist_ids())

In [4]:
def get_block_ids(d_id):
    """Returns a list of block ids."""
    block_muni = 'http://aahar.jharkhand.gov.in/dealer_user_logs/getBlockMunicipality'
    dist_post = {'data[DealerUserLog][district_id]':d_id}
    r = requests.post(block_muni, data=dist_post)
    content = r.text
    soup = BeautifulSoup(content, "html.parser")
    block_ids = [(option['value'],option.text) for option in soup.find_all('option', value=True)[1:]]
    #print(block_ids)
    return block_ids

In [5]:
#Want data from 15-8-2016 to 30-11-2016
def get_dates():
    numdays = 109
    base = datetime.date.today()
    date_list = [(base - datetime.timedelta(days=x)).strftime("%d-%m-%Y") for x in range(0, numdays)]
    return date_list

In [6]:
def get_dealer_links(block_post):
    r = requests.post("http://aahar.jharkhand.gov.in/dealer_user_logs/hhdLoginDetail", data=block_post)
    #data = r.text
    soup = BeautifulSoup(r.text, "html.parser")
    main_table = soup.find('table', id='maintable')
    tmp_link_list = []
    for link in main_table.find_all('a', href=True):
        tmp_link_list.append(link['href'])
    return tmp_link_list
    

In [7]:
def get_transactions(base_url, dealer_link_list, d_id,block_key, block_value, dt):
    """Returns transactions."""
    
    for link in dealer_link_list:
        if urllib.urlopen(base_url+link).read().decode('utf-8').find("SORRY!!!! No record found....") == -1:
            tmp_data = pd.read_html(base_url+link)[0][1:]
            tmp_data.columns = tmp_data.iloc[0]
            tmp_data = tmp_data[1:] #take the data less the header row  
            tmp_data['district_id'] = pd.Series(d_id, index=tmp_data.index)
            tmp_data['date'] = pd.Series(dt, index=tmp_data.index)
            tmp_data['block_name'] = pd.Series(block_value, index=tmp_data.index)
            tmp_data['block_id'] = pd.Series(block_key, index=tmp_data.index)
        
            if os.path.isfile('./test.csv') == False:
                tmp_data.to_csv('test.csv', encoding='utf-8')
            else:
                with open('test.csv', 'a') as f:
                    tmp_data.to_csv(f, header=False, encoding='utf-8')
        
        #return data





In [8]:
#List for dealer links
#dealer_link_list = []
base_url = "http://aahar.jharkhand.gov.in"
dist_ids = list(range(1,25))

for d_id in dist_ids:
    b_ids = get_block_ids(d_id)
    for key, value in b_ids:
        date_list = get_dates()[1:3]
        for dt in date_list: 
            #print(dt)
            block_post = {'data[DealerUserLog][district_id]': d_id,
                      'data[DealerUserLog][block_city_id]': key,
                      'data[DealerUserLog][stockdatefrom]': dt }
            #print(block_post)
            #dealer_link_list.append(get_dealer_links(block_post))
            get_transactions(base_url, get_dealer_links(block_post), d_id,key, value, dt)
            
    #need a function that makes requests and processes responses, then calls function to build csv 
            
#print(dealer_links_list[1:10])


    

KeyboardInterrupt: 

In [8]:
#def append_csv(data):
#    with open('foo.csv', 'a') as f:
#             (data).to_csv(f, header=False)
            
            
def write_csv(data):
    # we need to test to see if file exists first
    # we need to create a file if it doesn't exist 
    # come up with naming schema - best to prob. use dates parameters used in query
    # we need to add the dealers (the dealer name is in hindi in the table), district, block and other identifiers
    # below is the basic writing out operation for a row list, we'll need to change it some to make it
    # work with the above and then appending new rows
    csvFile = open("files/transactions.csv", 'wt')
    writer = csv.writer(csvFile)
    #try:
        for row in data:
           csvRow = []
            for cell in row.findAll(['td', 'th']):
                csvRow.append(cell.get_text())
            writer.writerow(csvRow)
    #finally:
        csvFile.close()
    pass #tell python not to run function, remove when you want to run it
    
    

    

IndentationError: unexpected indent (<ipython-input-8-210c8040bd87>, line 16)

In [ ]:
link = "/transactions/hhdLoginTransaction/NGYwN2U0Y2YtZThjNC00OWUxLTg1ZGQtMGVkODBhODY3OTFi/MQ==/MjAxNi0xMC0xNw=="
#print(link)
base_url = "http://aahar.jharkhand.gov.in"
if urllib.urlopen(base_url+link).read().decode('utf-8').find("SORRY!!!! No record found....") != -1:
    print('TRUE')
#print(base_url+link)
web = urllib.urlopen(base_url+link)
data = web.read()
print(data.decode().find("SORRY!!!! No record found...."))
#soup = BeautifulSoup(data, "html.parser")
#main_table = soup.find('table', id='maintable')
#print(main_table)
#print(main_table.th.get_text() for th in table.find("tr").find_all("th"))

link1 = "/transactions/hhdLoginTransaction/NGY2MmQ3ODYtYjFiOC00NDYwLTkzYzAtMGRmODBhODZiOTA0/MjQ=/MjAxNi0xMi0wOA=="
urllib.urlopen(base_url+link1).read().decode('utf-8').find("SORRY!!!! No record found....")
print(base_url+link1)
d_id = 1

data = pd.read_html(base_url+link1)[0][1:]
data.columns = data.iloc[0]
data = data[1:] #take the data less the header row

data['district_id'] = pd.Series(d_id, index=data.index)
data
#data[0][1:]



In [ ]:
link = "/transactions/hhdLoginTransaction/NGYwN2U0Y2YtZThjNC00OWUxLTg1ZGQtMGVkODBhODY3OTFi/MQ==/MjAxNi0xMC0xNw=="
#print(link)
base_url = "http://aahar.jharkhand.gov.in"
#print(base_url+link)
#web = urllib.urlopen(base_url+link)
#data = web.read()
#print(data.decode().find("SORRY!!!! No record found...."))
#soup = BeautifulSoup(data, "html.parser")
#main_table = soup.find('table', id='maintable')
#print(main_table)
#print(main_table.th.get_text() for th in table.find("tr").find_all("th"))

link1 = "/transactions/hhdLoginTransaction/NGY2MmQ3ODYtYjFiOC00NDYwLTkzYzAtMGRmODBhODZiOTA0/MjQ=/MjAxNi0xMi0wOA=="
print(base_url+link1)
data = pd.read_html(base_url+link1)

data[0]

#web = urllib.urlopen(base_url+link1)
#data = web.read()
#soup = BeautifulSoup(data, "html.parser")
#soup = BeautifulSoup(data, "lxml")

#soup.prettify()
#main_table = soup.find('table', id='maintable')
#print(main_table.find_all('tr')[7:])
#print(main_table)

data = {
    'obs_number' : [],
    'dealer_name' : [],
    'day': [],
    'date_time' : [],
    'ration_card_number' : [],
    'present_cardtype' : [],
    'old_cardtype' : [],
    'hh_uid' : [],
    'member_name' : [],
    'rice_kg' : [],
    'wheat_kg' : [],
    'kerosene_lit' : [],
    'salt_kg' : []
    
    
}

#rows = [tr.findAll('td') for tr in main_table.findAll('tr')[7:]]
rows = main_table.find_all('tr')[2:]

for row in rows:
    cols = row.find_all('td')


#cols = row.find_all('td')
    data['obs_number'].append( cols[0].get_text() )
    data['dealer_name'].append( cols[1].get_text() )
    data['date_time'].append( cols[2].get_text() )
    data['ration_card_number'].append( cols[3].get_text() )
    data['present_cardtype'].append( cols[4].get_text() )
    data['old_cardtype'].append( cols[5].get_text() )
    data['hh_uid'].append( cols[6].get_text() )
    data['member_name'].append( cols[7].get_text() )
    data['rice_kg'].append( cols[8].get_text() )
    data['wheat_kg'].append( cols[9].get_text() )
    data['kerosene_lit'].append( cols[10].get_text() )
    data['salt_kg'].append( cols[11].get_text() )

#print(data)
#import numpy
for key,values in data.iteritems():
    print(key)
    for value in values:
        print(value)
    

#data = pd.DataFrame( data )
#dogData.to_csv("AKC_Dog_Registrations.csv")


#print(rows)
#th are table headers variable names
#tr are new rows
#td are the data cells

In [ ]:
print(tmp_link_list[1])

In [47]:
#print(dealer_links_list[1:10])

for links in tmp_link_list:
    print(links['href'])

/transactions/hhdLoginTransaction/NGYwN2UzMTYtNmU1MC00M2RjLTg0Y2MtMGVkODBhODY3OTFi/MQ==/MjAxNi0xMC0xNw==
/transactions/hhdLoginTransaction/NGYwN2U0Y2YtZThjNC00OWUxLTg1ZGQtMGVkODBhODY3OTFi/MQ==/MjAxNi0xMC0xNw==
/transactions/hhdLoginTransaction/NGYwN2U2NjUtMzU4OC00MDEyLTgzMGItMGVkODBhODY3OTFi/MQ==/MjAxNi0xMC0xNw==
/transactions/hhdLoginTransaction/NGYwN2ViNDItZmNmOC00ZWE5LWEyZjgtMGVkODBhODY3OTFi/MQ==/MjAxNi0xMC0xNw==
/transactions/hhdLoginTransaction/NGYwN2YwOTQtN2EwNC00YTQwLTk2YjEtMGVkODBhODY3OTFi/MQ==/MjAxNi0xMC0xNw==
/transactions/hhdLoginTransaction/NGYwN2Y0YjYtNjhlYy00ZjEwLTk2ZWQtMGVkODBhODY3OTFi/MQ==/MjAxNi0xMC0xNw==
/transactions/hhdLoginTransaction/NGYwN2Y3YmQtNGNhMC00MWIxLWEwMWUtMGVkODBhODY3OTFi/MQ==/MjAxNi0xMC0xNw==
/transactions/hhdLoginTransaction/NGYwN2ZhNTAtZWMzMC00MDI0LTk5YmUtMGVkODBhODY3OTFi/MQ==/MjAxNi0xMC0xNw==
/transactions/hhdLoginTransaction/NGYwN2ZiOTgtZjg2MC00Mzc5LWFlY2MtMGVkODBhODY3OTFi/MQ==/MjAxNi0xMC0xNw==
/transactions/hhdLoginTransaction/NGYwYWJlNzQtMzA1MC00O

In [ ]:
whos

In [ ]:
def get_transactions1(base_url, dealer_link_list, d_id, block_value, dt):
    """Returns transactions."""
    for link in dealer_link_list:
        #data = pd.read_html(base_url+link1)
        data = urllib.urlopen(base_url+link).read()
        if data.decode().find("SORRY!!!! No record found....") != -1:
            continue
        soup = BeautifulSoup(data, "lxml")
        main_table = soup.find('table', id='maintable')
        rows = main_table.find_all('tr')[2:]
        
        data = {
            'district': []
            'block': []
            'date': []
            'obs_number' : [],
            'dealer_name' : [],
            'date_time' : [],
            'ration_card_number' : [],
            'present_cardtype' : [],
            'old_cardtype' : [],
            'hh_uid' : [],
            'member_name' : [],
            'rice_kg' : [],
            'wheat_kg' : [],
            'kerosene_lit' : [],
            'salt_kg' : []
        }
        
        for row in rows:
            cols = row.find_all('td')
            data['district'].append( d_id )
            data['block'].append( block_value )
            data['date'].append( dt )
            data['obs_number'].append( cols[0].get_text() )
            data['dealer_name'].append( cols[1].get_text() )
            data['date_time'].append( cols[2].get_text() )
            data['ration_card_number'].append( cols[3].get_text() )
            data['present_cardtype'].append( cols[4].get_text() )
            data['old_cardtype'].append( cols[5].get_text() )
            data['hh_uid'].append( cols[6].get_text() )
            data['member_name'].append( cols[7].get_text() )
            data['rice_kg'].append( cols[8].get_text() )
            data['wheat_kg'].append( cols[9].get_text() )
            data['kerosene_lit'].append( cols[10].get_text() )
            data['salt_kg'].append( cols[11].get_text() )

    return data
    pass

